# Example usage of segmentation to btrack to napari visualization

This example uses TIF files saved out from segmentation using *stardist3D*, although will work for other segmentation pipelines too.

In [1]:
import os

import btrack
import imageio
import napari

import numpy as np

In [2]:
PATH = "/media/quantumjot/Data/Test"

### method 1 - using a numpy array

In this example, each image from the timelapse is a 3D volume (32 x 1200 x 1200) and there are 10 timepoints

In [3]:
def segmentation_arr(path):
    """Segmentation as numpy array."""
    
    stack = []
    files = os.listdir(path)
    files.sort()
    
    for filename in files:
        print(filename)
        
        # note, change this to imageio.imread for 2D data
        img = imageio.volread(os.path.join(path, filename))
        stack.append(img)
        
    return np.stack(stack, axis=0)

In [4]:
stack = segmentation_arr(PATH)

labels_0.tif
labels_1.tif
labels_2.tif
labels_3.tif
labels_4.tif
labels_5.tif
labels_6.tif
labels_7.tif
labels_8.tif
labels_9.tif


Now we print out the shape of the stack (T, Z, Y, X):

In [5]:
stack.shape

(10, 32, 1200, 1200)

### Method 2 - using a generator

This is useful if you're resource constrained and don't want to load all of the image data, or they are stored in an unusual format

In [6]:
def segmentation_generator(path):
    """Segmentation generator"""
    files = os.listdir(path)
    files.sort()
    
    for filename in files:
        
        # note, change this to imageio.imread for 2D data
        img = imageio.volread(os.path.join(path, filename))
        
        yield img

In [7]:
generator = segmentation_generator(PATH)

## localizing the objects

Now we use a utility function to localise the objects in the segmentation, and also apply anisotropic scaling (using the `scale` option, here the z-values are scaled by 2x)

In [8]:
obj_from_arr = btrack.utils.segmentation_to_objects(stack, properties=('area', ), scale=(2., 1., 1.))

[INFO][2021/04/19 12:46:47 PM] Localizing objects from segmentation...
[INFO][2021/04/19 12:46:53 PM] Objects are of type: <class 'dict'>
[INFO][2021/04/19 12:46:53 PM] ...Found 11625 objects in 10 frames.


In [9]:
# inspect the first object
obj_from_arr[0]

,ID,x,y,z,t,dummy,states,label,prob,area
0,0,939.0,20.0,12.0,0,False,0,5,0.0,1381


In [ ]:
obj_from_generator = btrack.utils.segmentation_to_objects(
    generator, 
    properties = ('area', 'major_axis_length'), 
    scale=(2., 1., 1.)
)

[INFO][2021/04/19 12:46:53 PM] Localizing objects from segmentation...


In [ ]:
# inspect the first object
obj_from_generator[0]

## run btrack with the objects

We will use the objects from the generator here.

In [ ]:
# initialise a tracker session using a context manager
with btrack.BayesianTracker() as tracker:

    # configure the tracker using a config file
    tracker.configure_from_file('../models/cell_config.json')
    tracker.max_search_radius = 10

    # append the objects to be tracked
    tracker.append(obj_from_generator)

    # set the volume
    tracker.volume=((0, 1200), (0, 1200), (-1e5, 64.))

    # track them (in interactive mode)
    tracker.track_interactive(step_size=100)

    # generate hypotheses and run the global optimizer
    tracker.optimize()

    tracker.export(os.path.join(PATH, 'tracking.h5'), obj_type='obj_type_1')

    # get the tracks in a format for napari visualization
    data, properties, graph = tracker.to_napari(ndim=3)
    
    tracks = tracker.tracks

## visualize with napari

Note that we also set the scale of the images here to account for the anisotropy.

In [ ]:
viewer = napari.Viewer()
viewer.add_labels(stack, scale=(1., 2., 1., 1.), name='Segmentation')
viewer.add_tracks(data, properties=properties, graph=graph, name='Tracks')